# Merging Data
Steps to consider:
* Want to use weighted similarity
    * multiply the similarity score of each ingredient pair by its corresponding weight based on the relative positions of the ingredients in their respective lists
    * Sum up the weighted similarity scores across all ingredient pairs to obtain an overall similarity score between the two products
    * Assign higher weights to ingredients that are closer together, and higher weights if they are closer to 1 in the list
* Need to match ingredients to their place in the ingredient_info data set

In [2]:
import pandas as pd
import re as re

In [3]:
ingredients = pd.read_csv("data/ingredient_info_processed.csv")
products = pd.read_csv("data/skincare_products_processed.csv")

In [4]:
products

,Label,Brand,Name,Price,Rank,Combination,Dry,Normal,Oily,Sensitive,Ingredient_Placement,Ingredient
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,1,Algae (Seaweed) Extract
1,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,2,Mineral Oil
2,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,3,Petrolatum
3,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,4,Glycerin
4,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,5,Isohexadecane
...,...,...,...,...,...,...,...,...,...,...,...,...
45811,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,15,Alpha-isomethyl Ionone
45812,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,16,CI 14700 (Red 4)
45813,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,17,CI 19140 (Yellow 5)
45814,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,18,CI 42090 (Blue 1).


In [72]:
products[products['Name'] == "Crème de la Mer"].head(10)

,Label,Brand,Name,Price,Rank,Combination,Dry,Normal,Oily,Sensitive,Ingredient_Placement,Ingredient
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,1,Algae (Seaweed) Extract
1,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,2,Mineral Oil
2,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,3,Petrolatum
3,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,4,Glycerin
4,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,5,Isohexadecane
5,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,6,Microcrystalline Wax
6,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,7,Lanolin Alcohol
7,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,8,Citrus Aurantifolia (Lime) Extract
8,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,9,Sesamum Indicum (Sesame) Seed Oil
9,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,10,Eucalyptus Globulus (Eucalyptus) Leaf Oil


In [6]:
def ingredient_in_INCI(inci, ingredient_name):
    cleaned_inci_text = re.sub(r'[^\w\s]', ' ', inci).lower()
    words_in_inci = set(cleaned_inci_text.split())
    cleaned_ingredient_text = re.sub(r'[^\w\s]', ' ', ingredient_name).lower()
    words_in_name = set(cleaned_ingredient_text.split())

    return (words_in_name <= words_in_inci)


def get_shortest_INCI(df, ingredient_name):
    filtered_df = \
        df[df['INCI name'].apply(lambda x: ingredient_in_INCI(x, ingredient_name))]
    
    if filtered_df.size == 0:
        ingredient_name_clean = re.sub(r'\([^)]*\)', '', ingredient_name)

        filtered_df = \
            df[df['INCI name'].apply(lambda x: ingredient_in_INCI(x, ingredient_name_clean))]
        
        if filtered_df.size == 0:
            return filtered_df
        
    shortest_index = filtered_df['INCI name'].str.len().idxmin()
    return filtered_df.loc[[shortest_index]]
    

In [84]:
get_shortest_INCI(ingredients, "Algae (Seaweed) Extract")

,COSING Ref No,INCI name,INN name,Ph. Eur. Name,CAS No,EC No,Chem/IUPAC Name / Description,Restriction,Function,Update Date
1475,54290,ALGAE EXTRACT,NaN,NaN,92128-82-0 / 68917-51-1,295-780-4 / -,Algae Extract is an extract of various species...,NaN,"FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITIO...",15/10/2010


In [14]:
get_shortest_INCI(ingredients, "CI 14700 (Red 4)")['Function'].iloc[0]

'COLORANT'